In [1]:
import pandas as pd
import numpy as np

In [2]:
data_df = pd.read_csv('example_dataframe_100_recipes.csv')
data_df.head()

,Unnamed: 0,cuisine_SP,aisle_SP,ingredients_SP
0,recipe_title,[],"['Produce', 'Milk, Eggs, Other Dairy', 'Spices...","['cauliflower', 'egg', 'italian seasoning', 'm..."
1,Dreamy Chai Rice Pudding,[],"['Pasta and Rice', 'Milk, Eggs, Other Dairy', ...","['long grain rice', 'whole milk', 'tea', 'cinn..."
2,Instant Pot Chicken Tacos,['Mexican'],"['Spices and Seasonings', 'Meat', 'Produce', '...","['black pepper', 'chicken breasts', 'lime juic..."
3,Homemade King Ranch Chicken Casserole,[],"['Milk, Eggs, Other Dairy', 'Meat', 'Canned an...","['butter', 'cooked chicken breasts', 'cream of..."
4,Mangoes with Rum and Ice Cream,[],"['Produce', 'Baking', 'Alcoholic Beverages', '...","['mangoes', 'sugar', 'dark rum', 'lime juice',..."


In [3]:
len(data_df)

101

In [43]:
total_ingredient_list = []
row_list = []
for row in data_df['ingredients_SP']:
    row = row.replace("[", '')
    row = row.replace("'", '')
    row = row.replace("]", '')
    row = row.replace('"', '')
    row_list = str.split(row, ',')
    for element in row_list:
        total_ingredient_list.append(element)
print(total_ingredient_list)

['cauliflower', ' egg', ' italian seasoning', ' monterey jack cheese', ' pepper', ' salt', 'long grain rice', ' whole milk', ' tea', ' cinnamon powder', ' ginger powder', ' pea pods', ' whole star anise', ' cloves', ' vanilla essence', ' brown sugar', 'black pepper', ' chicken breasts', ' lime juice', ' purple onion', ' salsa', ' taco seasoning', 'butter', ' cooked chicken breasts', ' cream of chicken soup', ' cream of mushroom soup', ' diced tomatoes with green chilis', ' flour tortillas', ' jack cheese', ' onion', 'mangoes', ' sugar', ' dark rum', ' lime juice', ' lime rind', ' vanilla ice cream', 'bone in chicken thighs', ' yellow onion', ' garlic cloves', ' fresh thyme', ' scotch bonnet pepper', ' powdered ginger', ' curry powder', ' coconut oil', ' cilantro', ' scallion', ' vegetable oil', ' yellow onion', ' garlic cloves', ' curry powder', ' amchar masala', ' cumin', ' salt', ' water', ' potatoes', 'canned garbanzo beans', ' chili powder', ' coriander', ' cumin', ' flour', ' fres

In [44]:
total_ingredients_df = pd.DataFrame(total_ingredient_list)

In [45]:
total_ingredient_df = total_ingredients_df.rename(columns = {0:  'ingredient'})

In [52]:
total_ingredient_df['count'] = 1

In [53]:
total_ingredient_df

,ingredient,count
0,cauliflower,1
1,egg,1
2,italian seasoning,1
3,monterey jack cheese,1
4,pepper,1
...,...,...
1023,egg,1
1024,italian seasoning,1
1025,monterey jack cheese,1
1026,pepper,1


In [54]:
total_ingredient_counts = total_ingredient_df.groupby(['ingredient']).agg({'count': 'count'})

In [55]:
total_ingredient_counts.reset_index()

,ingredient,count
0,Salt & Pepper,2
1,acorn squash,1
2,active yeast,1
3,agave nectar,1
4,all purpose flour,1
...,...,...
486,skinless boneless chicken breast,1
487,water,2
488,whipping cream,1
489,white cake mix,1
